In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import data
from data import en_chset, ko_chset_cho, ko_chset_jung, ko_chset_jong
%matplotlib inline
plt.rcParams['image.cmap'] = 'Greys'

print("packs loaded")
%connect_info

packs loaded
{
  "signature_scheme": "hmac-sha256",
  "kernel_name": "",
  "iopub_port": 42494,
  "control_port": 49793,
  "ip": "127.0.0.1",
  "transport": "tcp",
  "shell_port": 36074,
  "stdin_port": 38867,
  "hb_port": 50517,
  "key": "ad423625-d8e3-44ed-9322-fc57a78f1a28"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-14f332f4-8666-4807-a7a1-f474c81c894d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
index_data, img, label = data.get_all('data/161020.tgz')

tar opened for index
tar opened for image
 0% complete (1 / 300000)
 3% complete (10001 / 300000)
 7% complete (20001 / 300000)
10% complete (30001 / 300000)
13% complete (40001 / 300000)
17% complete (50001 / 300000)
20% complete (60001 / 300000)
23% complete (70001 / 300000)
27% complete (80001 / 300000)
30% complete (90001 / 300000)
33% complete (100001 / 300000)
37% complete (110001 / 300000)
40% complete (120001 / 300000)
43% complete (130001 / 300000)
47% complete (140001 / 300000)
50% complete (150001 / 300000)
53% complete (160001 / 300000)
57% complete (170001 / 300000)
60% complete (180001 / 300000)
63% complete (190001 / 300000)
67% complete (200001 / 300000)
70% complete (210001 / 300000)
73% complete (220001 / 300000)
77% complete (230001 / 300000)
80% complete (240001 / 300000)
83% complete (250001 / 300000)
87% complete (260001 / 300000)
90% complete (270001 / 300000)
93% complete (280001 / 300000)
97% complete (290001 / 300000)
image loaded
label loaded


In [3]:
class BatchBuffer:
    def __init__(self, offset, size):
        self.offset = offset
        self.size = size
        self.index = 0
    
    def read(self, num=None):
        raise NotImplementedError('Unimplemented')
        
    def seek(self, num, mode=0):
        if mode == 0:
            self.index = num
        elif mode == 1:
            self.index += num
        elif mode == 2:
            self.index = self.size+num
        else:
            raise NotImplementedError('Unimplemented')
        
    def tell(self):
        return self.index

class ArrayBuffer(BatchBuffer):
    def __init__(self, array, offset, size):
        if size < 0:
            size = len(array) - offset
        super().__init__(offset, size)
        self.array = array
    
    def read(self, num=None):
        if self.index >= self.size:
            return None
        
        if num is None:
            num = self.size - self.index
            
        ni = self.index+num
        if ni >= self.size:
            ni = self.size
        ret = self.array[self.offset+self.index:self.offset+ni]
        self.index += num
        return ret

trainimg = ArrayBuffer(img, 15000, -1)
trainlabel = ArrayBuffer(label, 15000, -1)
testimg = ArrayBuffer(img, 0, 12000)
testlabel = ArrayBuffer(label, 0, 12000)
cvimg = ArrayBuffer(img, 12000, 3000)
cvlabel = ArrayBuffer(label, 12000, 3000)
randidx = np.random.randint(trainimg.size, size=2)

print("data loaded")

data loaded


In [4]:
def flatten_cnn(layer):
    layer_shape = layer.get_shape().as_list()
    n_out = layer_shape[1] * layer_shape[2] * layer_shape[3]
    return tf.reshape(layer, [-1, n_out])

def build_nn(shape, X, name):
    n_before = int(X.get_shape()[1])
    W = tf.Variable(tf.truncated_normal([n_before, shape], stddev=0.1), name=name+"_W")
    b = tf.Variable(tf.constant(0.1, shape=[shape]), name=name+"_b")
    return tf.matmul(X, W)+b

def build_cnn(cnn_shape, patch_shape, X, name, stride=1):
    n_before = int(X.get_shape()[3])
    W = tf.Variable(tf.truncated_normal([patch_shape[0], patch_shape[1], n_before, cnn_shape], stddev=0.1),
                   name=name+"_W")
    b = tf.Variable(tf.constant(0.1, shape=[cnn_shape]), name=name+"_b")
    layer = tf.nn.relu(tf.nn.conv2d(X, W, strides=[1, stride, stride, 1], padding='SAME') + b)
    return layer

def max2d_pool(layer):
    return tf.nn.max_pool(layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def slice_label(tf_label, len_tuple):
    cur = 0
    sliced = []
    for l in len_tuple:
        sliced.append(tf.slice(tf_label, [0, cur], [-1, l]))
        cur += l
    return tuple(sliced)

print("function loaded")

function loaded


In [12]:
Y_size = len(ko_chset_cho)+len(ko_chset_jung)+len(ko_chset_jong)+len(en_chset)+4 

tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 32, 32])
Y = tf.placeholder(tf.float32, [None, Y_size])
keep_prob = tf.placeholder(tf.float32)

Y_cho, Y_jung, Y_jong, Y_en = slice_label(Y,
                                         (len(ko_chset_cho)+1,
                                         len(ko_chset_jung)+1,
                                         len(ko_chset_jong)+1,
                                         len(en_chset)+1))
# Small inception model
# http://laonple.blog.me/220704822964
X_reshape = tf.reshape(X, [-1, 32, 32, 1])
cnn_1_5 = build_cnn(12, [5,5], X_reshape, "cnn_1_5")
cnn_1_3 = build_cnn(36, [3,3], X_reshape, "cnn_1_3")
cnn_1_concat = tf.concat(3, [cnn_1_5, cnn_1_3])
cnn_1_pool = max2d_pool(cnn_1_concat) # 16 * 16 * 48

cnn_2_5 = build_cnn(18, [5,5], cnn_1_pool, "cnn_2_5")
cnn_2_3 = build_cnn(48, [3,3], cnn_1_pool, "cnn_2_3")
cnn_2_1 = build_cnn(30, [1,1], cnn_1_pool, "cnn_2_1")
cnn_2_concat = tf.concat(3, [cnn_2_5, cnn_2_3, cnn_2_1])
cnn_2_pool = max2d_pool(cnn_2_concat) # 8 * 8 * 96

cnn_3_5_reduce = build_cnn(18, [1,1], cnn_2_pool, "cnn_3_5_reduce")
cnn_3_5 = build_cnn(36, [5,5], cnn_3_5_reduce, "cnn_3_5")
cnn_3_3_reduce = build_cnn(64, [1,1], cnn_2_pool, "cnn_3_3_reduce")
cnn_3_3 = build_cnn(96, [3,3], cnn_3_3_reduce, "cnn_3_3")
cnn_3_1 = build_cnn(60, [1,1], cnn_2_pool, "cnn_3_1")
cnn_3_concat = tf.concat(3, [cnn_3_5, cnn_3_3, cnn_3_1])
cnn_3_pool = max2d_pool(cnn_3_concat) # 4 * 4 * 192

dense_1 = tf.nn.relu(build_nn(1024, flatten_cnn(cnn_3_pool), "dense_1"))
dropout_1 = tf.nn.dropout(dense_1, keep_prob)

logit = build_nn(Y_size, dropout_1, "logit")
logit_cho, logit_jung, logit_jong, logit_en = slice_label(logit,
                                         (len(ko_chset_cho)+1,
                                         len(ko_chset_jung)+1,
                                         len(ko_chset_jong)+1,
                                         len(en_chset)+1))
h_cho = tf.nn.softmax(logit_cho)
h_jung = tf.nn.softmax(logit_jung)
h_jong = tf.nn.softmax(logit_jong)
h_en = tf.nn.softmax(logit_en)

learning_rate = tf.placeholder(tf.float32)
cost_cho = tf.nn.softmax_cross_entropy_with_logits(logit_cho, Y_cho)
cost_jung = tf.nn.softmax_cross_entropy_with_logits(logit_jung, Y_jung)
cost_jong = tf.nn.softmax_cross_entropy_with_logits(logit_jong, Y_jong)
cost_en = tf.nn.softmax_cross_entropy_with_logits(logit_en, Y_en)
cost = cost_cho + cost_jung * 1.5 + cost_jong * 0.5 + cost_en
cost_mean = tf.reduce_mean(cost) # mean of batch set

var_before_adam = tf.all_variables()

train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_cho = tf.equal(tf.argmax(Y_cho,1), tf.argmax(h_cho,1))
correct_jung = tf.equal(tf.argmax(Y_jung,1), tf.argmax(h_jung,1))
correct_jong = tf.equal(tf.argmax(Y_jong,1), tf.argmax(h_jong,1))
correct_two = tf.logical_or(tf.logical_and(correct_cho, tf.logical_or(correct_jung, correct_jong)),
                           tf.logical_and(correct_jung, correct_jong))
correct_ko = tf.logical_and(tf.logical_and(correct_cho, correct_jung), correct_jong)
correct_en = tf.equal(tf.argmax(Y_en,1), tf.argmax(h_en,1))
correct_all = tf.logical_and(correct_ko, correct_en)
accuracy = tf.reduce_mean(tf.cast(correct_all, tf.float32))
accuracy_two = tf.reduce_mean(tf.cast(correct_two, tf.float32))
accuracy_cho = tf.reduce_mean(tf.cast(correct_cho, tf.float32))
accuracy_jung = tf.reduce_mean(tf.cast(correct_jung, tf.float32))
accuracy_jong = tf.reduce_mean(tf.cast(correct_jong, tf.float32))
accuracy_ko = tf.reduce_mean(tf.cast(correct_ko, tf.float32))
accuracy_en = tf.reduce_mean(tf.cast(correct_en, tf.float32))

sess = tf.Session()
print("session loaded")

session loaded


In [13]:
def init_session():
    sess.run(tf.initialize_all_variables())
    print("session initialized")

def get_accuracy_no_batch(imgset, labelset):
    return sess.run((accuracy, accuracy_cho, accuracy_jung, accuracy_jong, accuracy_two, accuracy_en),
                                        feed_dict={X:imgset, Y:labelset, keep_prob:1})

def get_accuracy(imgbuf, labelbuf, batch=True):
    imgbuf.seek(0)
    labelbuf.seek(0)
    
    if batch:
        temp_tuple = 0, 0, 0, 0, 0, 0
        i = 0
        while(True):
            batch_x = imgbuf.read(100)
            batch_y = labelbuf.read(100)
            if batch_x is None:
                break
            temp_tuple = tuple([item1 + item2 * batch_x.shape[0] for item1, item2 in
                                zip(temp_tuple,
                                    get_accuracy_no_batch(batch_x, batch_y))])
            i += batch_x.shape[0]
        tacc, tacc_cho, tacc_jung, tacc_jong, tacc_two, tacc_en = tuple([item / i for item in temp_tuple])
    else:
        tacc, tacc_cho, tacc_jung, tacc_jong, tacc_two, tacc_en = get_accuracy_no_batch(imgbuf.read(), labelbuf.read())
    return tacc, tacc_cho, tacc_jung, tacc_jong, tacc_two, tacc_en
    
def print_accuracy(imgbuf, labelbuf, batch=True):
    tacc, tacc_cho, tacc_jung, tacc_jong, tacc_two, tacc_en = get_accuracy(imgbuf, labelbuf, batch)
    print ("overall accuracy = %.3f                          " % tacc)
    print ("two of three = %.3f" % tacc_two)
    print ("cho = %.3f" % tacc_cho)
    print ("jung = %.3f" % tacc_jung)
    print ("jong = %.3f" % tacc_jong)
    print ("en = %.3f" % tacc_en)

def do_training(is_console=False, lr_init = 0.003):
    trainsize = trainimg.size
    batchsize = 100
    batch_per_epoch = int(trainsize/batchsize)
    print ("Training %d, mini-batch %d * %d" % (trainsize, batchsize, batch_per_epoch))
    epoch = 0

    i = 0
    lr = lr_init
    while (epoch < 4):
        if i % 200 == 0 :
            tacc = get_accuracy(cvimg, cvlabel, True)[0]
            print ("%6dth epoch : cv accuracy = %.3f                  " % (epoch, tacc))

        batch_x = trainimg.read(batchsize)
        batch_y = trainlabel.read(batchsize)
        
        if batch_x is None:
            print_accuracy(testimg, testlabel, True)
            trainimg.seek(0)
            trainlabel.seek(0)
            epoch += 1
            continue
        
        cur_cost = sess.run((train, cost_mean),
                            feed_dict={X:batch_x, Y:batch_y, keep_prob:0.5, learning_rate:lr})[1]
        if(is_console):
            print ("%dth... lr = %.2e, cost = %.2e\r" % (i, lr, cur_cost), end="")
        lr = lr * (1 - 0.0003)
        i += 1
        
    print("train complete--------------------------------")
    print("test accuracy ---")
    print_accuracy(testimg, testlabel, True)
    print("train accuracy ---")
    print_accuracy(trainimg, trainlabel, True)
    
def error_check(chset, pred_tf, label_tf, imgbuf, labelbuf):
    label_len = label_tf.get_shape()[1]
    n_error = np.zeros([label_len, label_len])
    n_all = np.zeros(label_len)
    new_chset = chset + ["inv"]
    
    batchsize = 100
    
    imgbuf.seek(0)
    labelbuf.seek(0)
    while(True):
        batch_x = imgbuf.read(batchsize)
        batch_y = labelbuf.read(batchsize)
        if batch_x is None:
            break
        batchsize = batch_x.shape[0]
        h, y = sess.run((pred_tf, label_tf), feed_dict={X:batch_x, Y:batch_y, keep_prob:1})
        for j in range(batchsize):
            n_all[np.argmax(y[j])] += 1
            if (np.argmax(h[j]) != np.argmax(y[j])):
                n_error[np.argmax(y[j])][np.argmax(h[j])] += 1

    print ("Error rate")
    for i, ch in enumerate(new_chset):
        most_error = np.argmax(n_error[i])
        print ("%s : %2.0f%% (%4d / %4d)" %
               (ch, float(np.sum(n_error[i])) / n_all[i] * 100, np.sum(n_error[i]), n_all[i]), end="")
        if n_error[i][most_error] > 0:
            print ("%6d errors with %s" % (n_error[i][most_error], new_chset[most_error]))
        else:
            print ("")
            
def save_ckpt(path):
    saver = tf.train.Saver(var_before_adam)
    saver.save(sess, path)
    print("ckpt saved")
    
def load_ckpt(path):
    saver = tf.train.Saver(var_before_adam)
    saver.restore(sess, path)
    print("ckpt loaded")
    
def train_and_save(path):
    init_session()
    do_training(True)
    save_ckpt(path)
            
print("tf function loaded")

tf function loaded


In [12]:
print_accuracy(testimg, testlabel, True)

overall accuracy = 0.996                          
two of three = 1.000
cho = 1.000
jung = 0.997
jong = 0.999
en = 0.999


In [13]:
error_check(en_chset, h_en, Y_en, testimg, testlabel)

Error rate
0 :  0% (   0 /   41)
1 :  3% (   1 /   32)     1 errors with l
2 :  0% (   0 /   35)
3 :  0% (   0 /   28)
4 :  0% (   0 /   40)
5 :  0% (   0 /   44)
6 :  0% (   0 /   29)
7 :  0% (   0 /   33)
8 :  0% (   0 /   48)
9 :  0% (   0 /   39)
a :  0% (   0 /   36)
b :  0% (   0 /   34)
c :  0% (   0 /   33)
d :  0% (   0 /   27)
e :  0% (   0 /   36)
f :  0% (   0 /   29)
g :  0% (   0 /   35)
h :  0% (   0 /   41)
i :  0% (   0 /   32)
j :  0% (   0 /   32)
k :  0% (   0 /   29)
l :  9% (   4 /   46)     4 errors with I
m :  0% (   0 /   38)
n :  0% (   0 /   37)
o :  0% (   0 /   34)
p :  0% (   0 /   33)
q :  0% (   0 /   36)
r :  0% (   0 /   36)
s :  0% (   0 /   31)
t :  0% (   0 /   34)
u :  0% (   0 /   39)
v :  0% (   0 /   41)
w :  0% (   0 /   38)
x :  3% (   1 /   38)     1 errors with X
y :  0% (   0 /   40)
z :  0% (   0 /   32)
A :  0% (   0 /   23)
B :  0% (   0 /   30)
C :  0% (   0 /   35)
D :  0% (   0 /   30)
E :  0% (   0 /   26)
F :  0% (   0 /   42)
G :  

In [14]:
error_check(ko_chset_jung, h_jung, Y_jung, testimg, testlabel)

Error rate
ㅏ :  0% (   1 /  946)     1 errors with ㅑ
ㅐ :  0% (   1 /  644)     1 errors with ㅓ
ㅑ :  0% (   0 /  290)
ㅒ :  0% (   0 /   57)
ㅓ :  0% (   0 /  782)
ㅔ :  1% (   5 /  527)     3 errors with ㅖ
ㅕ :  0% (   0 /  476)
ㅖ :  1% (   2 /  160)     2 errors with ㅔ
ㅗ :  1% (   5 /  671)     2 errors with ㅡ
ㅘ :  0% (   1 /  340)     1 errors with ㅏ
ㅙ :  0% (   0 /  168)
ㅚ :  0% (   1 /  394)     1 errors with ㅟ
ㅛ :  0% (   1 /  248)     1 errors with ㅗ
ㅜ :  0% (   1 /  677)     1 errors with ㅗ
ㅝ :  0% (   1 /  233)     1 errors with ㅟ
ㅞ :  1% (   1 /  153)     1 errors with ㅝ
ㅟ :  0% (   1 /  419)     1 errors with ㅚ
ㅠ :  0% (   0 /  342)
ㅡ :  1% (   5 /  702)     3 errors with ㅜ
ㅢ :  0% (   0 /  110)
ㅣ :  1% (   5 /  706)     2 errors with ㅓ
inv :  0% (   0 / 2955)
